In [ ]:
import os
import sys
import pandas as pd
import math

os.chdir('./scripts')

import load_config
import matplotlib.pyplot as plt
import sweetviz as sv
import numpy as np
import seaborn as sns
import datetime

In [ ]:
os.getcwd()

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [ ]:
config = load_config.load_config_const()
plt.style.use('seaborn')

In [ ]:
config['files_dgt']

# Estudio del Dataset de transacciones de turismos de la DGT

El presente notebook tiene como objetivo hace un estudio de las características relevantes del dataset de transferencias de turismos.

## 1. Carga de datos

Debido al pandas, al convertir los datos en un dataframe, el campo de código postal es convertido en tipo númerico float o integer. Es necesario reconstruir los códigos.

In [ ]:
def cod_postal_zero_int_repair(cod_postal):
    if (not math.isnan( cod_postal )):
        cod_postal = int(cod_postal)
        if ( len(str(cod_postal)) == 5 ):
            return str(cod_postal)
        elif ( len(str(int(cod_postal))) == 4 ):
            return '0'+str(int(cod_postal))
        else:
            return int(cod_postal)
    else:
        return cod_postal

La siguiente función se usa para obtener el código de provincia del código postal

In [ ]:
def return_id_province_by_zip(cod_postal):
    if (cod_postal):
        cod_postal = str(cod_postal)
        return cod_postal[0:2]
    else:
        return cod_postal

is_weekend se usa para obtener si el día del traspaso es fin de semana, ya que, como se verá más adelante, estos días los traspasos de vehículos son mínimos. Interesante conocer esto para implementar la serie temporal y obtener una predición más fiable.

In [ ]:
def is_weekend(day_week):
    if (day_week == 'Saturday' or day_week == 'Sunday'):
        return True
    else:
        return False

Crea una correspondencia de código de provincia y su nombre

In [ ]:
def return_province(id_province):
    list_ids_province = {
        '01': 'Álava',
        '08': 'Barcelona',
        '15': 'La Coruña',
        '23': 'Jaén',
        '30': 'Murcia',
        '38': 'Santa Cruz de Tenerife',
        '45': 'Toledo',
        '02': 'Albacete',
        '09': 'Burgos',
        '16': 'Cuenca',
        '24': 'León',
        '31': 'Navarra',
        '46': 'Valencia',
        '03': 'Alicante',
        '10': 'Cáceres',
        '17': 'Gerona',
        '25': 'Lérida',
        '32': 'Orense',
        '39': 'Cantabria',
        '47': 'Valladolid',
        '04': 'Almería',
        '11': 'Cádiz',
        '18': 'Granada',
        '26': 'La Rioja',
        '33': 'Asturias',
        '40': 'Segovia',
        '48': 'Vizcaya',
        '05': 'Ávila',
        '12': 'Castellón',
        '19': 'Guadalajara',
        '34': 'Palencia',
        '41': 'Sevilla',
        '49': 'Zamora',
        '06': 'Badajoz',
        '13': 'Ciudad Real',
        '20': 'Guipúzcoa',
        '27': 'Lugo',
        '35': 'Las Palmas',
        '42': 'Soria',
        '50': 'Zaragoza',
        '07': 'Islas Baleares',
        '21': 'Huelva',
        '28': 'Madrid',
        '36': 'Pontevedra',
        '43': 'Tarragona',
        '51': 'Ceuta',
        '14': 'Córdoba',
        '22': 'Huesca',
        '29': 'Málaga',
        '37': 'Salamanca',
        '44': 'Teruel',
        '52': 'Melilla'
    }
    
    return list_ids_province.get(id_province)

Debido a que algunas marcas se dan de alta con distinto nombre, es requerido una forma de unificar las marcas y así ser más preciso a la hora de contabilizar los traspasos por marca

In [ ]:
def return_make(make):
    list_makes = {
        'ABARTH': 'ABARTH',
        'ALFA': 'ALFA ROMEO',
        'ALFA ROMEO': 'ALFA ROMEO',
        'ASTON MARTIN': 'ASTON MARTIN',
        'AUDI': 'AUDI',
        'BENTLEY': 'BENTLEY',
        'BMW': 'BMW',
        'BMW I': 'BMW',
        'CADILLAC': 'CADILLAC',
        'CHEVROLET': 'CHEVROLET',
        'CHRYSLER': 'CHRYSLER',
        'CITROEN': 'CITROEN',
        'CUPRA': 'CUPRA',
        'DACIA': 'DACIA',
        'DAEWOO': 'DAEWOO',
        'DAIHATSU': 'DAIHATSU',
        'DAIMLER': 'DAIMLER',
        'DAIMLER AG': 'DAIMLER',
        'DAIMLER CHRYSLER': 'CHRYSLER',
        'DODGE': 'DODGE',
        'DS': 'DS',
        'FERRARI': 'FERRARI',
        'FIAT': 'FIAT',
        'FORD': 'FORD',
        'FORD CNG TECHNIK': 'FORD',
        'FORD-CNG-TECHNIK': 'FORD',
        'GMC': 'GMC',
        'HONDA': 'HONDA',
        'HUMMER': 'HUMMER',
        'HYUNDAI': 'HYUNDAI',
        'INFINITI': 'INFINITI',
        'ISUZU': 'ISUZU',
        'IVECO': 'IVECO',
        'JAGUAR': 'JAGUAR',
        'JAGUAR LAND ROVER LIMIT': 'LAND-ROVER',
        'JEEP': 'JEEP',
        'KIA': 'KIA',
        'LADA': 'LADA',
        'LAMBORGHINI': 'LAMBORGHINI',
        'LANCIA': 'LANCIA',
        'LAND ROVER': 'LAND-ROVER',
        'LEXUS': 'LEXUS',
        'LOTUS': 'LOTUS',
        'M.G': 'MG',
        'MAHINDRA': 'MAHINDRA',
        'MASERATI': 'MASERATI',
        'MAZDA': 'MAZDA',
        'MCC SMART': 'SMART',
        'MCLAREN': 'MCLAREN',
        'MERCEDES': 'MERCEDES-BENZ',
        'MERCEDES-AMG': 'MERCEDES-BENZ',
        'MERCEDES-BENZ': 'MERCEDES-BENZ',
        'MG': 'MG',
        'MICRO COMPACT CAR': 'SMART',
        'MINI': 'MINI',
        'MITSUBISHI': 'MITSUBISHI',
        'NISSAN': 'NISSAN',
        'OPEL': 'OPEL',
        'PEUGEOT': 'PEUGEOT',
        'PORSCHE': 'PORSCHE',
        'QUATTRO': 'AUDI',
        'RANGE ROVER': 'ROVER',
        'RENAULT': 'RENAULT',
        'ROVER': 'ROVER',
        'SAAB': 'SAAB',
        'SANTANA': 'SANTANA',
        'SEAT': 'SEAT',
        'SEAT FIAT': 'SEAT',
        'SKODA': 'SKODA',
        'SMART': 'SMART',
        'SSANGYONG': 'SSANGYONG',
        'SUBARU': 'SUBARU',
        'SUZUKI': 'SUZUKI',
        'SUZUKI SANTANA': 'SANTANA',
        'TATA': 'TATA',
        'TESLA': 'TESLA',
        'TESLA MOTORS': 'TESLA',
        'TOYOTA': 'TOYOTA',
        'VOLKSWAGEN': 'VOLKSWAGEN',
        'VOLKSWAGEN V W': 'VOLKSWAGEN',
        'VOLVO': 'VOLVO',
    }
    
    return list_makes.get(make)

In [ ]:
def return_avaliable_makes():
    return [
        'ABARTH',
        'ALFA',
        'ALFA ROMEO',
        'ASTON MARTIN',
        'AUDI',
        'BENTLEY',
        'BMW',
        'BMW I',
        'CADILLAC',
        'CHEVROLET',
        'CHRYSLER',
        'CITROEN',
        'CUPRA',
        'DACIA',
        'DAEWOO',
        'DAIHATSU',
        'DAIMLER',
        'DAIMLER AG',
        'DAIMLER CHRYSLER',
        'DODGE',
        'DS',
        'FERRARI',
        'FIAT',
        'FORD',
        'FORD CNG TECHNIK',
        'FORD-CNG-TECHNIK',
        'GMC',
        'HONDA',
        'HUMMER',
        'HYUNDAI',
        'INFINITI',
        'ISUZU',
        'IVECO',
        'JAGUAR',
        'JAGUAR LAND ROVER LIMIT',
        'JEEP',
        'KIA',
        'LADA',
        'LAMBORGHINI',
        'LANCIA',
        'LAND ROVER',
        'LEXUS',
        'LOTUS',
        'M.G',
        'MAHINDRA',
        'MASERATI',
        'MAZDA',
        'MCC SMART',
        'MCLAREN',
        'MERCEDES',
        'MERCEDES-AMG',
        'MERCEDES-BENZ',
        'MG',
        'MICRO COMPACT CAR',
        'MINI',
        'MITSUBISHI',
        'NISSAN',
        'OPEL',
        'PEUGEOT',
        'PORSCHE',
        'QUATTRO',
        'RANGE ROVER',
        'RENAULT',
        'ROVER',
        'SAAB',
        'SANTANA',
        'SEAT',
        'SEAT FIAT',
        'SKODA',
        'SMART',
        'SSANGYONG',
        'SUBARU',
        'SUZUKI',
        'SUZUKI SANTANA',
        'TATA',
        'TESLA',
        'TESLA MOTORS',
        'TOYOTA',
        'VOLKSWAGEN',
        'VOLKSWAGEN V W',
        'VOLVO'
    ]

Este el proceso principal donde se carga el dataframe con todos los archivos de la dgt por años. Al ser un número muy alto de registros, se accede a una fracción de los mismos. Igualmente, se realiza una serie de procedimientos para limpìar campos, estandarizarlos o eliminar aquellos valorer Nan que no pueden rellenarse con otros valores

In [ ]:
columns = config['fields_dgt_process']


df = '';

for dgt_file in config['files_dgt']:

    route_file = config['base_route'] + config['data_route'] +  config['data_dgt_route'] + '/' + dgt_file
    
    print(f'Processing {dgt_file}')
    
    df_aux = pd.read_csv(route_file, compression='gzip', sep=config['csv_dgt_separator'], low_memory=False)
    df_aux.columns = columns
    
    
    df_aux = df_aux.sample(frac=0.4)
    
    
    df_aux['FEC_MATRICULA'] = pd.to_datetime(df_aux['FEC_MATRICULA'])
    df_aux['FEC_TRAMITACION'] = pd.to_datetime(df_aux['FEC_TRAMITACION'])
    df_aux['FEC_TRAMITE'] = pd.to_datetime(df_aux['FEC_TRAMITE'])
    df_aux['FEC_PRIM_MATRICULACION'] = pd.to_datetime(df_aux['FEC_PRIM_MATRICULACION'])
    
    
    df_aux = df_aux[df_aux['MARCA_ITV'].isin( return_avaliable_makes() )]
    df_aux['MARCA_ITV'] = df_aux['MARCA_ITV'].apply(lambda x: return_make(x) )
    df_aux = df_aux[df_aux['MARCA_ITV'] != '']
    
    
    df_aux['Q_query'] = df_aux['Q_query'].apply(lambda x: int(x) if not math.isnan(x) else x )
    df_aux['year_Q'] = df_aux['year_Q'].apply(lambda x: int(x) if not math.isnan(x) else x )
    df_aux['CODIGO_POSTAL'] = df_aux['CODIGO_POSTAL'].apply(lambda x: int(x) if not math.isnan(x) else x )
    df_aux['CLAVE_TRAMITE'] = df_aux['CLAVE_TRAMITE'].apply(lambda x: int(x) if not math.isnan(x) else x )
    df_aux['NUM_TRANSMISIONES'] = df_aux['NUM_TRANSMISIONES'].apply(lambda x: int(x) if not math.isnan(x) else x )
    df_aux['NUM_PLAZAS'] = df_aux['NUM_PLAZAS'].apply(lambda x: int(x) if not math.isnan(x) else x )
    df_aux['NUM_TITULARES'] = df_aux['NUM_TITULARES'].apply(lambda x: int(x) if not math.isnan(x) else x )
    df_aux['CILINDRADA_ITV'] = df_aux['CILINDRADA_ITV'].apply(lambda x: int(x) if not math.isnan(x) else x )
    df_aux['COD_MUNICIPIO_INE_VEH'] = df_aux['COD_MUNICIPIO_INE_VEH'].apply(lambda x: int(x) if not math.isnan(x) else x )
    # df_aux['KW_ITV'] = df_aux['KW_ITV'].apply(lambda x: int(x) if not math.isnan(x) else x )
    df_aux['NUM_PLAZAS_MAX'] = df_aux['NUM_PLAZAS_MAX'].apply(lambda x: int(x) if not math.isnan(x) else x )
    df_aux['CO2_ITV'] = df_aux['CO2_ITV'].apply(lambda x: int(x) if not math.isnan(x) else x )
    
    df_aux['CODIGO_POSTAL'] = df_aux['CODIGO_POSTAL'].apply(lambda x: cod_postal_zero_int_repair(x) )
    df_aux['COD_MUNICIPIO_INE_VEH'] = df_aux['COD_MUNICIPIO_INE_VEH'].apply(lambda x: cod_postal_zero_int_repair(x) )
    df_aux['ID_MUNICIPIO'] = df_aux['COD_MUNICIPIO_INE_VEH'].apply(lambda x: return_id_province_by_zip(x) )
    
    df_aux = df_aux[df_aux['FEC_TRAMITACION'] >= '2015-01-01 00:00:00']
    df_aux = df_aux[df_aux['FEC_TRAMITACION'] <= '2021-12-31 23:59:59']
    
    df_aux = df_aux[df_aux['FEC_TRAMITE'] >= '2015-01-01 00:00:00']
    df_aux = df_aux[df_aux['FEC_TRAMITE'] <= '2021-12-31 23:59:59']
    
    df_aux['ID_MUNICIPIO'].fillna(value=np.nan, inplace=True)
    
    df_aux = df_aux[df_aux['ID_MUNICIPIO'].notna()]
    df_aux = df_aux[df_aux['year_Q'].notna()]
    
    df_aux = df_aux[df_aux['ID_MUNICIPIO'] != 0]
    
    df_aux['year_Q'] = df_aux['year_Q'].apply(lambda x: int(x) if not math.isnan(x) else x )
    df_aux['Q_query'] = df_aux['Q_query'].apply(lambda x: int(x) if not math.isnan(x) else x )

    df_aux['month_year_Q'] = df_aux['FEC_TRAMITE'].dt.month
    
    index = df_aux.index
    number_of_rows = len(index)
    print(f'### Lines -> {number_of_rows}')
    
    if (type(df) == pd.core.frame.DataFrame):
        print('### Merge with main datagrame')
        df = pd.concat([df,df_aux]);
    else:
        print('### Create main datagrame')
        
        df = df_aux.copy()
        
    del df_aux
    
    index = df.index
    number_of_rows = len(index)
    print(f'### Total lines -> {number_of_rows}')

In [ ]:
df.sample(10)

Comprobamos el tamaño del dataframe obtenido luego del proceso

In [ ]:
len(df.index)

Así como comprobamos los tipos de las columnas

In [ ]:
df.dtypes

Analizamos los campos, tanto su matriz de correlación como el contenido de cada uno de los campos con la herramienta "sweetviz". Nos permite sacar un dossier de información muy completo ahorrando la implementación de código para ello

In [ ]:
my_report = sv.analyze(df)
my_report.show_notebook()

## 2. Analisis de campos

### 2.1 Fecha de tramite

Mediante la agrupación de la fecha de tramite, se obtendrá el número de traspasos diarios. Importante para la obtención de la serie temporal más adelante

In [ ]:
df.groupby(['FEC_TRAMITE']).size().reset_index(name='counts')

Mostramos mediante una gráfica el las ventas por fecha, pudiendo reconocer a simple vista una serie de patrones

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
grouped = df.groupby(['FEC_TRAMITE']).size().reset_index(name='counts')

sns.lineplot(data=grouped, x='FEC_TRAMITE', y='counts')

La primera característica apreciable es que cada fin de año, el número de traspasos aumenta exponencialmente. Esto puede ser debido tanto a las fiestas, pagas extraordinarias o a la finalización de algún tipo de año fiscal, pero es algo frecuente todos los años.

En la época de verano, más concretamente en agosto, también se puede apreciar algo que se repite todos los años, una pequeña bajada en los traspasos de vehículos. No es una bajada importante, pero si es caracteristica del periodo.

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
grouped = df.groupby(['FEC_TRAMITE']).size().reset_index(name='counts')

plt.plot(
    grouped[(grouped['FEC_TRAMITE'] >= '2015-01-01') & (grouped['FEC_TRAMITE'] < '2015-12-31')]['FEC_TRAMITE'],
    grouped[(grouped['FEC_TRAMITE'] >= '2015-01-01') & (grouped['FEC_TRAMITE'] < '2015-12-31')]['counts']
)

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
grouped = df.groupby(['FEC_TRAMITE']).size().reset_index(name='counts')

plt.plot(
    grouped[(grouped['FEC_TRAMITE'] >= '2016-01-01') & (grouped['FEC_TRAMITE'] < '2016-12-31')]['FEC_TRAMITE'],
    grouped[(grouped['FEC_TRAMITE'] >= '2016-01-01') & (grouped['FEC_TRAMITE'] < '2016-12-31')]['counts']
)

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
grouped = df.groupby(['FEC_TRAMITE']).size().reset_index(name='counts')

plt.plot(
    grouped[(grouped['FEC_TRAMITE'] >= '2017-01-01') & (grouped['FEC_TRAMITE'] < '2017-12-31')]['FEC_TRAMITE'],
    grouped[(grouped['FEC_TRAMITE'] >= '2017-01-01') & (grouped['FEC_TRAMITE'] < '2017-12-31')]['counts']
)

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
grouped = df.groupby(['FEC_TRAMITE']).size().reset_index(name='counts')

plt.plot(
    grouped[(grouped['FEC_TRAMITE'] >= '2018-01-01') & (grouped['FEC_TRAMITE'] < '2018-12-31')]['FEC_TRAMITE'],
    grouped[(grouped['FEC_TRAMITE'] >= '2018-01-01') & (grouped['FEC_TRAMITE'] < '2018-12-31')]['counts']
)

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
grouped = df.groupby(['FEC_TRAMITE']).size().reset_index(name='counts')

plt.plot(
    grouped[(grouped['FEC_TRAMITE'] >= '2019-01-01') & (grouped['FEC_TRAMITE'] < '2019-12-31')]['FEC_TRAMITE'],
    grouped[(grouped['FEC_TRAMITE'] >= '2019-01-01') & (grouped['FEC_TRAMITE'] < '2019-12-31')]['counts']
)

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
grouped = df.groupby(['FEC_TRAMITE']).size().reset_index(name='counts')

plt.plot(
    grouped[(grouped['FEC_TRAMITE'] >= '2020-01-01') & (grouped['FEC_TRAMITE'] <= '2020-12-31')]['FEC_TRAMITE'],
    grouped[(grouped['FEC_TRAMITE'] >= '2020-01-01') & (grouped['FEC_TRAMITE'] <= '2020-12-31')]['counts']
)

El año del comienzo de la pandemia es muy notable el periodo de la cuarentena, habiendo una bajada extraordinaria de

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
grouped = df.groupby(['FEC_TRAMITE']).size().reset_index(name='counts')

plt.plot(
    grouped[(grouped['FEC_TRAMITE'] >= '2021-01-01') & (grouped['FEC_TRAMITE'] <= '2021-12-31')]['FEC_TRAMITE'],
    grouped[(grouped['FEC_TRAMITE'] >= '2021-01-01') & (grouped['FEC_TRAMITE'] <= '2021-12-31')]['counts']
)

In [ ]:
grouped = df.groupby(['FEC_TRAMITE']).size().reset_index(name='counts')

grouped['day_of_year'] = grouped['FEC_TRAMITE'].dt.dayofyear
grouped['year'] = grouped['FEC_TRAMITE'].dt.year

fig, ax = plt.subplots(figsize=(20,10))

#2019 -> red
sns.lineplot(data=grouped[(grouped['FEC_TRAMITE'] >= '2019-01-01') & (grouped['FEC_TRAMITE'] <= '2019-12-31')], x='day_of_year', y='counts', color='red', label='2019')
#2020 -> blue
sns.lineplot(data=grouped[(grouped['FEC_TRAMITE'] >= '2020-01-01') & (grouped['FEC_TRAMITE'] <= '2020-12-31')], x='day_of_year', y='counts', color='blue', label='2020')
#2020 -> green
sns.lineplot(data=grouped[(grouped['FEC_TRAMITE'] >= '2021-01-01') & (grouped['FEC_TRAMITE'] <= '2021-12-31')], x='day_of_year', y='counts', color='green', label='2021')




Al comparar el año de la pandemia con el año anterior y posterior, es curioso que, salvo el confinamiento estricto, no se observa una bajada con despecto a los otros años a lo largo del resto del año 

La siguiente representación corresponde a una vista de las transacciones en la época del confinamiento restrictivo

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
grouped = df.groupby(['FEC_TRAMITE']).size().reset_index(name='counts')

plt.plot(
    grouped[(grouped['FEC_TRAMITE'] >= '2020-03-01') & (grouped['FEC_TRAMITE'] <= '2020-06-30')]['FEC_TRAMITE'],
    grouped[(grouped['FEC_TRAMITE'] >= '2020-03-01') & (grouped['FEC_TRAMITE'] <= '2020-06-30')]['counts'],
)


Al comparar el periodo con respecto al del año siguiente, así como la media en el mismo, se ve un decremento de un 50% de  las transacciones

In [ ]:

grouped = df.groupby(['FEC_TRAMITE']).size().reset_index(name='counts')

grouped['day_of_year'] = grouped['FEC_TRAMITE'].dt.dayofyear
grouped['year'] = grouped['FEC_TRAMITE'].dt.year

fig, ax = plt.subplots(figsize=(20,10))

#2021 -> blue
sns.lineplot(data=grouped[(grouped['FEC_TRAMITE'] >= '2021-03-01') & (grouped['FEC_TRAMITE'] <= '2021-06-30')], x='day_of_year', y='counts', color='blue', label='2019')
#2020 -> red
sns.lineplot(data=grouped[(grouped['FEC_TRAMITE'] >= '2020-03-01') & (grouped['FEC_TRAMITE'] <= '2020-06-30')], x='day_of_year', y='counts', color='red', label='2020')




plt.axhline(
    y = grouped[(grouped['FEC_TRAMITE'] >= '2020-03-01') & (grouped['FEC_TRAMITE'] <= '2020-06-30')]['counts'].describe()['mean'],
    color = 'r',
    linestyle = '--'
)

plt.axhline(
    y = grouped[(grouped['FEC_TRAMITE'] >= '2021-03-01') & (grouped['FEC_TRAMITE'] <= '2021-06-30')]['counts'].describe()['mean'],
    color = 'b',
    linestyle = '--'
)




In [ ]:
mean_2021 = grouped[(grouped['FEC_TRAMITE'] >= '2021-03-01') & (grouped['FEC_TRAMITE'] <= '2021-06-30')]['counts'].describe()['mean']
mean_2020 = grouped[(grouped['FEC_TRAMITE'] >= '2020-03-01') & (grouped['FEC_TRAMITE'] <= '2020-06-30')]['counts'].describe()['mean']


percent = mean_2020 * 100 /  mean_2021
percent

Al fijarnos en la distribución de transacciones de un mes cualquiera vemos que hay una frecuencia reseñable. En los fines de semana las transacciones tienden a 0, a casi no haber ninguna.

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
grouped = df.groupby(['FEC_TRAMITE']).size().reset_index(name='counts')

plt.plot(
    grouped[(grouped['FEC_TRAMITE'] >= '2021-03-01') & (grouped['FEC_TRAMITE'] <= '2021-03-31')]['FEC_TRAMITE'],
    grouped[(grouped['FEC_TRAMITE'] >= '2021-03-01') & (grouped['FEC_TRAMITE'] <= '2021-03-31')]['counts']
)

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
grouped = df.groupby(['FEC_TRAMITE']).size().reset_index(name='counts')

plt.plot(
    grouped[(grouped['FEC_TRAMITE'] >= '2021-03-07') & (grouped['FEC_TRAMITE'] <= '2021-03-15')]['FEC_TRAMITE'],
    grouped[(grouped['FEC_TRAMITE'] >= '2021-03-07') & (grouped['FEC_TRAMITE'] <= '2021-03-15')]['counts']
)

Si contabilizamos las ventas de todo el periodo del dataset, es más claro que los fines de semana apenas quedan registradas transacciones.

In [ ]:
df['day_of_week'] = df['FEC_TRAMITE'].dt.day_name()
df.groupby(['day_of_week']).size().reset_index(name='counts')

Por lo que, para facilitar la obtención de la serie temporal, es importante identificar los días que son fin de semana

In [ ]:
def is_weekend(day_week):
    if (day_week == 'Saturday' or day_week == 'Sunday'):
        return True
    else:
        return False

In [ ]:
df['day_of_week'].apply(lambda x: is_weekend(x))

In [ ]:
df['is_weekend'] = df['day_of_week'].apply(lambda x: is_weekend(x))

### 2.2 Marcas

Otra forma de centrar las transaciones son las marcas de los turismos.

In [ ]:
df.groupby(['MARCA_ITV']).size().reset_index(name='counts').sort_values('counts', ascending=False).head(10)

Podemos ver la evolución de las marcas de las que más traspasos se hacen y ver su evolución en el periodo

In [ ]:
df.groupby(['MARCA_ITV','year_Q']).size().reset_index(name='counts')

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
grouped = df.groupby(['MARCA_ITV','year_Q']).size().reset_index(name='counts')

sns.lineplot(data=grouped[grouped['MARCA_ITV'].isin(
        df.groupby(['MARCA_ITV']).size().reset_index(name='counts').sort_values('counts', ascending=False).head(10)['MARCA_ITV']
    ) ], x='year_Q', y='counts', hue='MARCA_ITV', legend='full')
ax.set_xlim([2015,2021])
ax.legend(title='MARCAS')

En el gráfico se puede apreciar la evolución de la evolución de las trasacciones por las marcas más "transaccionadas". Se puede ver como SEAT ha ido en aumento, superando al resto de marcas en 2019. También se comprueba la bajada de ventas en 2020 por el confinamiento.

### 2.3 Provincias

Se puede observar que Madrid se lleva la palma en lo que a transacciones de vehículos se refiere. Barcelona va por debajo por unos cuantos cientos de miles. Pero el salto importante se aprecia con respecto al tercer puesto, como se podrá ver mejor en la gráfica, la diferencia es de casi tres veces más.  

In [ ]:
df.groupby(['ID_MUNICIPIO']).size().reset_index(name='counts').sort_values('counts', ascending=False).head(10)

In [ ]:
df.groupby(['ID_MUNICIPIO','year_Q']).size().reset_index(name='counts').sort_values('counts', ascending=False)

In [ ]:
df['province'] = df['ID_MUNICIPIO'].apply(lambda x: return_province(x))

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))

grouped = df.groupby(['province','year_Q']).size().reset_index(name='counts').sort_values('counts', ascending=False)

sns.lineplot(data=grouped[grouped['province'].isin(
        df.groupby(['province']).size().reset_index(name='counts').sort_values('counts', ascending=False).head(10)['province']
    ) ], x='year_Q', y='counts', hue='province', legend='full')
ax.legend(title='PROVINCIAS')

### 2.4 Persona Juridica-Física

Esta columna hace referencia a que si el dueño del vehículo es una persona física o juridica. Este valor es ideal para comprobar si el destino del vehículo es una persona o una empresa

In [ ]:
df.groupby(['PERSONA_FISICA_JURIDICA']).size().reset_index(name='counts').sort_values('counts', ascending=False)

In [ ]:
df.groupby(['PERSONA_FISICA_JURIDICA']).size().reset_index(name='counts').sort_values('counts', ascending=False).plot.bar(x='PERSONA_FISICA_JURIDICA')

In [ ]:
df.groupby(['year_Q', 'PERSONA_FISICA_JURIDICA']).size().reset_index(name='counts')

In [ ]:
df.groupby(['year_Q','ID_MUNICIPIO','PERSONA_FISICA_JURIDICA']).size().reset_index(name='counts')

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
grouped = df.groupby(['year_Q', 'PERSONA_FISICA_JURIDICA']).size().reset_index(name='counts')

grouped['LITERAL_FISICA_JURIDICA'] = grouped['PERSONA_FISICA_JURIDICA'].apply(lambda x: 'Jurídica' if x == 'X' else 'Física')

sns.lineplot(data=grouped, x='year_Q', y='counts', hue='LITERAL_FISICA_JURIDICA', legend='full')

ax.legend(title='PERSONA FISICA JURIDICA')

Podemos ver una tendencia de unas marcas frente a otras dependiendo del tipo de persona

In [ ]:
grouped = df.groupby(['PERSONA_FISICA_JURIDICA','MARCA_ITV']).size().reset_index(name='counts')
grouped[grouped['PERSONA_FISICA_JURIDICA'] == 'D'].sort_values('counts', ascending=False).head(10)

In [ ]:
grouped = df.groupby(['PERSONA_FISICA_JURIDICA','MARCA_ITV']).size().reset_index(name='counts')
grouped[grouped['PERSONA_FISICA_JURIDICA'] == 'X'].sort_values('counts', ascending=False).head(10)

### 2.5 Número de Plazas

Este campo es desconcertante. El dataset está filtrado por vehículos de tipo turismo en el notebook previo a este, por lo que la variación de asientos es sorprendete. Aquí puede deberse a un factor humano en los casos más extremos, y ha turismos de carácteristicas especiales en otros. Es claro que la mayoría de turismos dispone de 5 plazas

In [ ]:
df.groupby(['NUM_PLAZAS']).size().reset_index(name='counts')

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
df.groupby(['NUM_PLAZAS']).size().reset_index(name='counts').plot.bar(ax=ax)

### 2.6 Cilindrada

Se define cilindrada como "Es la suma del volumen útil de todos los cilindros. Suele medirse en centímetros cúbicos (cm3), aunque también es usual su equivalencia en litros (2500 cm3 = 2,5 litros)". La mayoría de los vehículos gira en torno a los 1500-2000 cm3

In [ ]:
df.groupby(['CILINDRADA_ITV']).size().reset_index(name='counts')

In [ ]:
bins = [0, 10, 100, 500, 1000, 5000, 10000, 25000]

df['CILINDRADA_ITV'].value_counts(bins=bins, sort=False).reset_index()

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))

bins = [500, 1000, 1500, 2000, 2500, 3000, 5000]

df['CILINDRADA_ITV'].value_counts(bins=bins, sort=False).reset_index().plot.bar(x='index', y='CILINDRADA_ITV', rot=0, ax=ax)


### 2.7 Potencia

A diferencia de la cilindrada, la potencia de un turismo está más repartida

In [ ]:
df.groupby(['POTENCIA_ITV']).size().reset_index(name='counts')

In [ ]:
bins = [0, 10, 15, 25, 30, 35, 40, 45, 50]

df['POTENCIA_ITV'].value_counts(bins=bins, sort=False).reset_index()

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))

bins = [0, 10, 11, 12, 13, 14, 15, 25, 30, 35]

df['POTENCIA_ITV'].value_counts(bins=bins, sort=False).reset_index().plot.bar(x='index', y='POTENCIA_ITV', rot=0, ax=ax)


### 2.8 Emisiones CO2

Como el nombre indica, hace referencia a las emisiones de CO2 que expulsa el coche

In [ ]:
df.groupby(['CO2_ITV']).size().reset_index(name='counts')

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))

bins = [100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200]

df['CO2_ITV'].value_counts(bins=bins, sort=False).reset_index().plot.bar(x='index', y='CO2_ITV', rot=0, ax=ax)


Podemos diferenciar las marcas más contaminantes para hacer una comparativa. Es apreciable que los más contaminantes son marcas americanas o de lujo

In [ ]:
df.groupby(['MARCA_ITV'])['CO2_ITV'].agg('mean').reset_index().sort_values('CO2_ITV', ascending=False).head(10)

In [ ]:
 df.groupby(['MARCA_ITV'])['CO2_ITV'].agg('mean').reset_index().sort_values('CO2_ITV', ascending=False).head(10).plot.bar(x='MARCA_ITV')

### 2.9 Renting

Vehículo de renting o de compra tradicional. No es un campo relevante ya que en este país el renting es minoritario

In [ ]:
df.groupby(['RENTING']).size().reset_index(name='counts')

In [ ]:
df.groupby(['RENTING']).size().reset_index(name='counts').sort_values('counts', ascending=False).plot.bar(x='RENTING')

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
grouped = df.groupby(['year_Q', 'RENTING']).size().reset_index(name='counts')

grouped['LITERAL_RENTING'] = grouped['RENTING'].apply(lambda x: 'Nuevo' if x == 'N' else 'Renting')

sns.lineplot(data=grouped, x='year_Q', y='counts', hue='LITERAL_RENTING', legend='full')

ax.legend(title='Renting')

### 2.10 Homologación Europea

La inmensa mayoría de los vehículos de este dataset tiene categoría M1

In [ ]:
df.groupby(['CATEGORÍA_HOMOLOGACIÓN_EUROPEA_ITV']).size().reset_index(name='counts')

### 2.11 Nivel de emisiones

In [ ]:
df.groupby(['NIVEL_EMISIONES_EURO_ITV']).size().reset_index(name='counts').sort_values('counts', ascending=False).head(10)

In [ ]:
df.groupby(['NIVEL_EMISIONES_EURO_ITV']).size().reset_index(name='counts').sort_values('counts', ascending=False).head(10).plot.bar(x='NIVEL_EMISIONES_EURO_ITV')

### 2.12 Vehículo Eléctrico y autonomía

- PHEV Eléctrico Enchufable
- REEV Eléctrico de Autonomía Extendida
- HEV Eléctrico Híbrido
- BEV Eléctrico de Batería

In [ ]:
df.groupby(['CATEGORÍA_VEHÍCULO_ELÉCTRICO']).size().reset_index(name='counts')

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))


df.groupby(['CATEGORÍA_VEHÍCULO_ELÉCTRICO']).size().reset_index(name='counts').sort_values('counts', ascending=False).plot.bar(x='CATEGORÍA_VEHÍCULO_ELÉCTRICO', ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
grouped = df.groupby(['year_Q', 'CATEGORÍA_VEHÍCULO_ELÉCTRICO']).size().reset_index(name='counts')

sns.lineplot(data=grouped, x='year_Q', y='counts', hue='CATEGORÍA_VEHÍCULO_ELÉCTRICO', legend='full')

ax.legend(title='Categoría Vehículo Electrico')

# PHEV Eléctrico Enchufable
# REEV Eléctrico de Autonomía Extendida
# HEV Eléctrico Híbrido
# BEV Eléctrico de Batería

In [ ]:
df.groupby(['AUTONOMÍA_VEHÍCULO_ELÉCTRICO']).size().reset_index(name='counts')

In [ ]:
bins = [1000, 5000, 10000, 15000, 20000]

df['AUTONOMÍA_VEHÍCULO_ELÉCTRICO'].value_counts(bins=bins, sort=False).reset_index()

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))

bins = [1000, 5000, 10000, 15000, 20000]

df['AUTONOMÍA_VEHÍCULO_ELÉCTRICO'].value_counts(bins=bins, sort=False).reset_index().plot.bar(x='index', y='AUTONOMÍA_VEHÍCULO_ELÉCTRICO', rot=0, ax=ax)